In [7]:
import pandas as pd

from pulp import LpProblem, LpMaximize, lpSum, LpVariable, LpStatus, LpInteger, LpBinary

data = pd.read_excel("data_2024.xlsx").dropna()

#PLAYED_AT_LEAST = 30
#data = data[data['PLD'] >= PLAYED_AT_LEAST]

In [8]:
data

,Code,Name,Club,Value,Pts
0,101,D. Raya,ARS,3.9,40
1,102,A. Ramsdale,ARS,3.5,5
2,103,E. Martinez,AV,3.3,2
3,104,R. Olsen,AV,2.6,-8
4,105,J. Gauci,AV,2.5,0
...,...,...,...,...,...
478,696,M. Cunha,WOL,6.0,50
479,697,H. Hwang,WOL,5.9,42
480,698,E. Gonzalez Medina,WOL,5.7,0
481,699,J. Strand Larsen,WOL,5.5,0


In [13]:
def _position(player_code: int):
    match str(player_code)[0]: 
        case "1":
            return "GK"
        case "2":
            return "FB"
        case "3": 
            return "CB"
        case "4"|"5":
            return "MF"
        case "6" | "7":
            return "ST"

    raise ValueError(f"{player_code} is not recognized")

data['Position'] = data['Code'].apply(_position)

In [14]:
data.tail()

,Code,Name,Club,Value,Pts,Position
478,696,M. Cunha,WOL,6.0,50,ST
479,697,H. Hwang,WOL,5.9,42,ST
480,698,E. Gonzalez Medina,WOL,5.7,0,ST
481,699,J. Strand Larsen,WOL,5.5,0,ST
482,700,N. Fraser,WOL,5.5,0,ST


In [24]:
# Create the data groups

PLAYERS = data['Name'].tolist()

TEAMS = {
    team: data[data['Club'] == team]['Name'].tolist() for team in data['Club'].unique().tolist()
}

POSITIONS = {
    position: data[data['Position'] == position]['Name'].tolist() for position in data['Position'].unique().tolist()
}

values = {
    player: data[data['Name'] == player].iloc[0]['Value'] for player in PLAYERS
}

points = {
    player: data[data['Name'] == player].iloc[0]['Pts'] for player in PLAYERS
}

clubs_for_player = {
    row['Name']: row['Club'] for _, row in data.iterrows()
}

codes = {
     player: data[data['Name'] == player].iloc[0]['Code'] for player in PLAYERS
}


In [25]:
list(data['Club'].unique())

['ARS',
 'AV',
 'BHA',
 'BOU',
 'BRE',
 'CHE',
 'CP',
 'EVE',
 'FUL',
 'IPS',
 'LEI',
 'LIV',
 'MC',
 'MU',
 'NEW',
 'NFO',
 'SOT',
 'TOT',
 'WH',
 'WOL']

In [80]:
# Disallowed teams for defensive players
DISALLOWED_DF_TEAMS = ["BOU", "BRE", "CHE", "EVE", "FUL", "IPS", "LEI", "NFO", "SOT", "WOL"]
#ALLOWED_DF_TEAMS = ["MC", "LIV", "CHE", "ARS"]
DF_POSITIONS = ['GK', 'FB', "CB"]

# Disallowed players
DISALLOWED_PLAYERS = []

MUST_HAVE_PLAYERS = ["E. Smith Rowe"]

df_players = POSITIONS['FB'] + POSITIONS["CB"]
gk_players = POSITIONS['GK']
COMBINED_DF_PLAYERS = df_players + gk_players

ONE_OF_TEAMS = []
TWO_DEFENSIVE_PLAYER_TEAMS = ["ARS", "MC", "LIV"]


In [81]:
# Create problem

model = LpProblem("Fantasy", LpMaximize)

# variables
position_constraints = {
    'GK': 1,
    'FB': 2,
    "CB": 2,
    'MF': 4,
    'ST': 2
}

# Create player variables

player_vars = LpVariable.dicts("PlayerVariables", [(i) for i in PLAYERS], 0, 1, LpBinary)

## CONSTRAINTS

# Value constraint
model += lpSum(player_vars[(i)] * values[i] for i in PLAYERS) <= 50.0

# Team constraint
for team, players in TEAMS.items():
    model += lpSum(player_vars[(i)] for i in players) <= 2

# Disallowed defensive combinations
for team in TEAMS:
    if team in DISALLOWED_DF_TEAMS:
        model += lpSum(player_vars[(i)] for i in TEAMS[team] if i in COMBINED_DF_PLAYERS) == 0

# Only one defensive player per team to reduce correlation
for team in TEAMS:
   if team in TEAMS.keys() and team not in TWO_DEFENSIVE_PLAYER_TEAMS:
       model += lpSum(player_vars[(i)] for i in TEAMS[team] if i in COMBINED_DF_PLAYERS) <= 1

# Disallowed players
for player in DISALLOWED_PLAYERS:
    if player in PLAYERS:
        model += player_vars[(player)] == 0

# Must have players
for player in MUST_HAVE_PLAYERS:
    if player in PLAYERS:
        model += player_vars[(player)] == 1
    
# One of Teams
for team in ONE_OF_TEAMS:
    model += lpSum(player_vars[(i)] for i in TEAMS[team]) <= 1 

# Position constraint
for position, players in POSITIONS.items():
    model += lpSum(player_vars[(i)] for i in players) == position_constraints[position]
    
## OBJECTIVE FUNCTION
model += lpSum(player_vars[(i)] * points[i] for i in PLAYERS)


## SOLVE
model.solve()
    

1

In [82]:
LpStatus[model.status]

'Optimal'

In [83]:
total_sum = 0
for i in PLAYERS:
    if player_vars[(i)].varValue == 1:
        print(f'Code: {codes[i]}, Club: {clubs_for_player[i]}, Player: {i}, Points: {points[i]}, Value: {values[i]}')
        total_sum += points[i]
total_sum

Code: 116, Club: CP, Player: S. Johnstone, Points: 5, Value: 2.7
Code: 201, Club: ARS, Player: B. White, Points: 57, Value: 4.2
Code: 252, Club: MC, Player: J. Gvardiol, Points: 38, Value: 4.0
Code: 301, Club: ARS, Player: G. Magalhaes, Points: 54, Value: 4.0
Code: 362, Club: NEW, Player: J. Lascelles, Points: 9, Value: 2.9
Code: 447, Club: CHE, Player: C. Palmer, Points: 90, Value: 5.3
Code: 478, Club: FUL, Player: E. Smith Rowe, Points: 4, Value: 3.1
Code: 503, Club: MC, Player: P. Foden, Points: 73, Value: 4.9
Code: 550, Club: WH, Player: J. Bowen, Points: 60, Value: 4.8
Code: 606, Club: AV, Player: O. Watkins, Points: 93, Value: 7.2
Code: 675, Club: NEW, Player: A. Isak, Points: 71, Value: 6.8


554

In [85]:
588-554

34